In [1]:
# https://dacon.io/competitions/official/235930/codeshare/5508?page=1&dtype=recent


In [2]:
import random
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report, confusion_matrix

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [5]:
epochs = 100
lr = 1e-3
batch_size = 512
momentum = 0.9
weight_decay = 1e-4
seed = 42
NUM_WORKERS = 8
saved_model = '../saved/ae_embeding/best_model.pth'

param = {
            'epochs' : epochs,
            'lr' :lr,
            'batch_size' : batch_size,
            'momentum' : momentum,
            'weight_decay' : weight_decay
        }

In [6]:
def seed_everything(seed) :
    random.seed(seed)
    os.environ['PYHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed)    

In [7]:
train_df = pd.read_csv('../dataset/train.csv')
val_df = pd.read_csv('../dataset/val.csv')
train_df = train_df.drop(columns=['ID'])
val_df = val_df.drop(columns=['ID'])
print(train_df.shape)

(113842, 30)


In [8]:
# class CDataset(Dataset) :
#     def __init__(self, df, eval_mode=False) :
#         self.df = df
#         self.eval_mode = eval_mode
#         if self.eval_mode :
#             self.labels = self.df['Class']
#             self.df = self.df.drop(columns=['Class'])
        
#     def __getitem__(self, idx) :
#         if self.eval_mode :
#             self.x = self.df.iloc[idx, 1:]
#             self.y = self.labels[idx]
#             return torch.tensor(self.x, dtype=torch.float32),torch.tensor(self.y)
        
#         else :
#             self.x = self.df.iloc[idx, 1:]
#             return torch.tensor(self.x, dtype=torch.float32)
        
#     def __len__(self) :
#         return len(self.df)
        

In [9]:
class CDataset(Dataset):
    def __init__(self, df, eval_mode=False):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [10]:
train_dataset = CDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=NUM_WORKERS)

val_dataset = CDataset(val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, num_workers=NUM_WORKERS)

In [11]:
x = next(iter(train_loader))
x.shape

torch.Size([512, 30])

In [12]:
x, y  = next(iter(val_loader))
x.shape

torch.Size([512, 30])

In [13]:
class AutoEncoder(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.dim = 30
        self.embeding_dim = 1024
        self.hidden = 256
        self.act = nn.GELU()
        
        self.embeding = nn.Sequential(
            nn.Linear(self.dim,self.embeding_dim),
            nn.GELU(),
            nn.Linear(self.embeding_dim,self.hidden),
            nn.GELU(),
            nn.Linear(self.hidden,self.embeding_dim),
            # self.act
        )
        self.encoder = nn.Sequential(
            # nn.LayerNorm(self.embeding_dim),
            # nn.BatchNorm1d(self.embeding_dim),
            nn.Linear(self.embeding_dim,self.hidden),
            nn.BatchNorm1d(self.hidden),
            self.act,
            # nn.Linear(64,128),
            # nn.BatchNorm1d(128),
            # nn.LeakyReLU(),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.hidden,self.embeding_dim),
            nn.BatchNorm1d(self.embeding_dim),
            self.act,
            # nn.Linear(64,30),
        )
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)  
                
        
# #         freezing embeding layer        
#         for name, child in self.named_children() :
#             for param in child.parameters() :
#                 if name == 'embeding' :
#                     param.requires_grad = False                
        
    def forward(self, x) :
        x_ = self.embeding(x)
        x = self.encoder(x_)
        x = self.decoder(x)
        return x_, x
        

In [14]:
class Trainer() :
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device, **param) :
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.batch_size = param['batch_size']
        self.epochs = param['epochs']
        self.lr = param['lr']
        
        self.criterion = nn.L1Loss().to(device)
        
    def fit(self,) :
        self.model.to(self.device)
        best_score = 0
        for epoch in range(self.epochs) :
            self.model.train()
            train_loss = []
            
            for x in iter(self.train_loader) :
                x = x.to(self.device)
                x_, x = self.model(x)
                
                loss = self.criterion(x_, x)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                train_loss.append(loss.item())
                # self.scheduler.step()
                
            score = self.validation(self.model, 0.95)
            
            if self.scheduler is not None :
                self.scheduler.step(score)
            
            print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}]')
            for param_group in self.optimizer.param_groups:
                print(param_group['lr'])      
            
            # print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}] lr [{self.scheduler.get_lr()}]')

            if best_score < score :
                best_score = score
                torch.save(self.model.state_dict(), saved_model)
            
    def validation(self, eval_model, threshold) :
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred_y = []
        true_y = []
        
        with torch.no_grad():
            for x, y in iter(self.val_loader) :
                x = x.to(self.device)
                y = y.to(self.device)
                
                x_, x = self.model(x)
                diff = cos(x, x_).cpu().tolist()
                batch_pred = np.where(np.array(diff) < threshold, 1, 0).tolist()
                pred_y += batch_pred
                true_y += y.tolist()
                
        return f1_score(true_y, pred_y, average='macro')

In [15]:
model = AutoEncoder()

In [16]:
model = AutoEncoder()
model.eval()
optimizer = torch.optim.Adam(params=model.parameters(), lr = param['lr'])

# optimizer = torch.optim.SGD(model.parameters(), param['lr'],
#                             momentum=param['momentum'],
#                             weight_decay=param['weight_decay'])
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)
# scheduler = StepLR(optimizer, step_size=50, gamma=0.2)
scheduler = None

# scheduler = lr_scheduler.LambdaLR(optimizer, lambda epoch : 0.1 **(epoch //30))
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
#                                                 epochs              = param['epochs'], 
#                                                 steps_per_epoch     = int(len(train_dataset)/param['batch_size'])+1,
#                                                 max_lr              = param['lr'], 
#                                                 pct_start           = 0.05, 
#                                                 div_factor          = 5, 
#                                                 final_div_factor    = 5e+4)   


In [19]:
for param_group in optimizer.param_groups:
    print(param_group['lr'])

0.001


In [17]:
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device, **param)
# trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)

In [18]:
trainer.fit()

epoch :[0] train loss [0.044021268487978944] val score [0.011643347261451348]
0.001
epoch :[1] train loss [0.026487220993323977] val score [0.006202024658914319]
0.001
epoch :[2] train loss [0.025029654902794435] val score [0.5049338607994625]
0.001
epoch :[3] train loss [0.022255458380408887] val score [0.49584936695412385]
0.001
epoch :[4] train loss [0.01611600038825196] val score [0.009709313361527569]
0.001
epoch :[5] train loss [0.024050163560350647] val score [0.2537700272680949]
0.001
epoch :[6] train loss [0.013621002055637772] val score [0.4965062180473739]
0.001
epoch :[7] train loss [0.007662450843553305] val score [0.4975727726879557]
0.001
epoch :[8] train loss [0.0064365537847588545] val score [0.49844928455628396]
0.001
epoch :[9] train loss [0.005437482381761575] val score [0.49786528351152043]
0.001
epoch :[10] train loss [0.004868136980412152] val score [0.4984227685258613]
0.001
epoch :[11] train loss [0.004070885547054639] val score [0.4964349533801596]
0.001
epoch


KeyboardInterrupt



## inference

In [ ]:
model = AutoEncoder()
model.load_state_dict(torch.load(saved_model))
model.eval()

In [ ]:
test_df = pd.read_csv('../dataset/test.csv')
test_df = test_df.drop(columns=['ID'])
test_dataset = CDataset(test_df, eval_mode=False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
test_df.columns

In [ ]:
def prediction(model, threshold, test_loader, device):
    model.to(device)
    model.eval()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pred = []
    with torch.no_grad():
        for x in iter(test_loader):
            x = x.float().to(device)
            
            x_, x = model(x)
            
            diff = cos(x, x_).cpu().tolist()
            batch_pred = np.where(np.array(diff)<threshold, 1,0).tolist()
            pred += batch_pred
    return pred

In [ ]:
preds = prediction(model, 0.95, test_loader, device)

In [ ]:
submit = pd.read_csv('../dataset/sample_submission.csv')
submit['Class'] = preds
submit.to_csv('./submit_AE_embeding.csv', index=False)

In [ ]:
# submit.Class.value_counts()

In [ ]:
val_df = pd.read_csv('../dataset/val.csv')
val_y = val_df['Class'].values
val_df = val_df.drop(columns=['ID', 'Class'])
val_dataset = CDataset(val_df, eval_mode=False)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
# val_df.head()

In [ ]:
val_pred = prediction(model, 0.95, val_loader, device)

In [ ]:
f1_score(val_y, val_pred, average='macro')

In [ ]:
print(classification_report(val_y, val_pred))

In [ ]:
confusion_matrix(val_y, val_pred)

In [ ]:
answer = np.where(np.array(val_y) != np.array(val_pred))[0]
answer